In [363]:
import bs4, requests, re, bibtexparser

In [548]:
mode_layout = 'flat'
article_id = '3106246'
expformat = 'bibtex'

domain = 'https://dl.acm.org/'
article = 'citation.cfm?id={0}&'.format(article_id)
parametrs = 'preflayout={0}'.format(mode_layout)
url = domain + article + parametrs
# https://dl.acm.org/downformats.cfm?id=2492591&parent_id=&expformat=bibtex
url_download_bibtex = domain + 'downformats.cfm?id={0}&parent_id=&expformat={1}'.format(article_id, expformat)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
file = open('test.html', 'wb')

print(url, url_download_bibtex, sep='\n')

https://dl.acm.org/citation.cfm?id=3106246&preflayout=flat
https://dl.acm.org/downformats.cfm?id=3106246&parent_id=&expformat=bibtex


In [549]:
response = requests.get(url=url, headers=headers)
soup = bs4.BeautifulSoup(response.text, 'lxml')
bibtex_file = open('bibtex.bib', 'w').write(requests.get(url=url_download_bibtex, headers=headers).text)
bibtex_file = open('bibtex.bib', 'r')
bibtex_dict = bibtexparser.load(bibtex_file).entries

In [560]:
article_data = {}

article_data['title'] = bibtex_dict[0].get('title', None)
article_data['doi'] = bibtex_dict[0].get('doi', None) 
article_data['year'] = bibtex_dict[0].get('year', None)

if bibtex_dict[0].get('series', None):
    article_data['venue'] = bibtex_dict[0].get('series', None).split()[0]
else:
    article_data['venue'] = None

article_data, bibtex_dict

({'doi': '10.1145/3106237.3106246',
  'title': 'Why Modern Open Source Projects Fail',
  'venue': 'ESEC/FSE',
  'year': '2017'},
 [{'ENTRYTYPE': 'inproceedings',
   'ID': 'Coelho:2017:WMO:3106237.3106246',
   'acmid': '3106246',
   'address': 'New York, NY, USA',
   'author': 'Coelho, Jailton and Valente, Marco Tulio',
   'booktitle': 'Proceedings of the 2017 11th Joint Meeting on Foundations of Software Engineering',
   'doi': '10.1145/3106237.3106246',
   'isbn': '978-1-4503-5105-8',
   'keywords': 'GitHub, Open Source Software, Project failure',
   'location': 'Paderborn, Germany',
   'numpages': '11',
   'pages': '186--196',
   'publisher': 'ACM',
   'series': 'ESEC/FSE 2017',
   'title': 'Why Modern Open Source Projects Fail',
   'url': 'http://doi.acm.org/10.1145/3106237.3106246',
   'year': '2017'}])

In [551]:
divmain = soup.find('div', id='divmain')

authors_tags = divmain.find_all('td', {'style': 'padding-right:3px;', 'valign': 'top',  'nowrap': 'nowrap'})
authors = []

for author_tag in authors_tags:
    name = author_tag.text
    authors.append({'name': name.strip(),
                    'url': domain + author_tag.find('a')['href']})
    
article_data['authors'] = authors
article_data

{'authors': [{'name': 'Jailton Coelho',
   'url': 'https://dl.acm.org/author_page.cfm?id=99659190960&coll=DL&dl=ACM&trk=0'},
  {'name': 'Marco Tulio Valente',
   'url': 'https://dl.acm.org/author_page.cfm?id=81452608597&coll=DL&dl=ACM&trk=0'}],
 'doi': '10.1145/3106237.3106246',
 'title': 'Why Modern Open Source Projects Fail',
 'year': '2017'}

In [552]:
layout = soup.find('div', {'class': 'layout'})
flatbody = layout.find('div', {'class': 'flatbody'})

abstract = flatbody.text.strip()
article_data['abstract'] = abstract

article_data

{'abstract': 'Open source is experiencing a renaissance period, due to the appearance of modern platforms and workflows for developing and maintaining public code. As a result, developers are creating open source software at speeds never seen before. Consequently, these projects are also facing unprecedented mortality rates. To better understand the reasons for the failure of modern open source projects, this paper describes the results of a survey with the maintainers of 104 popular GitHub systems that have been deprecated. We provide a set of nine reasons for the failure of these open source projects. We also show that some maintenance practices---specifically the adoption of contributing guidelines and continuous integration---have an important association with a project failure or success. Finally, we discuss and reveal the principal strategies developers have tried to overcome the failure of the studied projects.',
 'authors': [{'name': 'Jailton Coelho',
   'url': 'https://dl.acm.